In [1]:
import os, pwd, sys, netifaces, socket

print(f"Hostname: {socket.gethostname()}, IP: {socket.gethostbyname(socket.gethostname())}")
print(f"\nNetwork interfaces: {netifaces.interfaces()}")
print([netifaces.ifaddresses(i) for i in netifaces.interfaces()])
print(f"\nCurrent user: {pwd.getpwuid(os.geteuid()).pw_name}, uid: {pwd.getpwuid(os.geteuid()).pw_uid}, shell: {pwd.getpwuid(os.geteuid()).pw_shell} ")
print(f"\nCurrent directory: {os.getcwd()}")
print("\nList current diretory: ",[i for i in os.listdir()])
print("\nPython vesion: ",sys.version.replace("\n"," "), "\n\nPython path: ",sys.path,"\n")

Hostname: 479ea4b9fba5, IP: 172.17.0.2

Network interfaces: ['lo', 'sit0', 'eth0']
[{17: [{'addr': '00:00:00:00:00:00', 'peer': '00:00:00:00:00:00'}], 2: [{'addr': '127.0.0.1', 'netmask': '255.0.0.0', 'peer': '127.0.0.1'}]}, {17: [{'addr': '00:00:00:00', 'broadcast': '00:00:00:00'}]}, {17: [{'addr': '02:42:ac:11:00:02', 'broadcast': 'ff:ff:ff:ff:ff:ff'}], 2: [{'addr': '172.17.0.2', 'netmask': '255.255.0.0', 'broadcast': '172.17.255.255'}]}]

Current user: root, uid: 0, shell: /bin/bash 

Current directory: /home/hlaz001/practicalnlp/Ch5

List current diretory:  ['06_EntityLinking-AzureTextAnalytics.ipynb', '02_NERTraining.ipynb', 'README.md', '04_NER_using_spaCy - CoNLL.ipynb', '05_BERT_CONLL_NER.ipynb', 'model', '08_Duckling.ipynb', '07_REWatson.ipynb', 'Data', '01_KPE.ipynb', '03_NERIssues.ipynb', 'spacyNER_data']

Python vesion:  3.8.5 (default, Jul 28 2020, 12:59:40)  [GCC 9.3.0] 

Python path:  ['/home/hlaz001/practicalnlp/Ch5', '/root/.vscode-server/extensions/ms-toolsai.jupyter-

# Training and Evaluating an NER model with spaCy on the CoNLL dataset

In this notebook, we will take a look at using spaCy commandline to train and evaluate a NER model. We will also compare it with the pretrained NER model in spacy. 

Note: we will create multiple folders during this experiment:
spacyNER_data 

## Step 1: Converting data to json structures so it can be used by Spacy

In [2]:
#Read the CONLL data from conll2003 folder, and store the formatted data into a folder spacyNER_data
#!mkdir spacyNER_data
#the above two lines create folders if they don't exist. If they do, the output shows a message that it
#already exists and cannot be created again
!python3.8 -m spacy convert "Data/conll2003/en/train.txt" spacyNER_data -c ner
!python3.8 -m spacy convert "Data/conll2003/en/test.txt" spacyNER_data -c ner
!python3.8 -m spacy convert "Data/conll2003/en/valid.txt" spacyNER_data -c ner

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (14987 documents): spacyNER_data/train.json
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (3684 documents): spacyNER_data/test.json
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (3466 documents): spacyNER_data/valid.json


#### For example, the data before and after running spacy's convert program looks as follows.

In [3]:
!echo "BEFORE : (Data/conll2003/en/train.txt)"
!head "Data/conll2003/en/train.txt" -n 11 | tail -n 9
!echo "\nAFTER : (Data/conll2003/en/train.json)"
!head "spacyNER_data/train.json" -n 64 | tail -n 49

BEFORE : (Data/conll2003/en/train.txt)
EU NNP B-NP B-ORG
rejects VBZ B-VP O
German JJ B-NP B-MISC
call NN I-NP O
to TO B-VP O
boycott VB I-VP O
British JJ B-NP B-MISC
lamb NN I-NP O
. . O O
\nAFTER : (Data/conll2003/en/train.json)
        ]
      }
    ]
  },
  {
    "id":1,
    "paragraphs":[
      {
        "sentences":[
          {
            "tokens":[
              {
                "orth":"EU",
                "tag":"NNP",
                "ner":"U-ORG"
              },
              {
                "orth":"rejects",
                "tag":"VBZ",
                "ner":"O"
              },
              {
                "orth":"German",
                "tag":"JJ",
                "ner":"U-MISC"
              },
              {
                "orth":"call",
                "tag":"NN",
                "ner":"O"
              },
              {
                "orth":"to",
                "tag":"TO",
                "ner":"O"
              },
              {
                "orth"

## Training the NER model with Spacy (CLI)

All the commandline options can be seen at: https://spacy.io/api/cli#train
We are training using the train program in spacy, for English (en), and the results are stored in a folder 
called "model" (created while training). Our training file is in "spacyNER_data/train.json" and the validation file is at: "spacyNER_data/valid.json". 

-G stands for gpu option.
-p stands for pipeline, and it should be followed by a comma separated set of options - in this case, a tagger and an NER are being trained simultaneously

In [4]:
!python3.8 -m spacy train en model spacyNER_data/train.json spacyNER_data/valid.json -G -p tagger,ner

⚠ Output directory is not empty
This can lead to unintended side effects when saving the model. Please use an
empty directory or a different path instead. If the specified output path
doesn't exist, the directory will be created for you.
Training pipeline: ['tagger', 'ner']
Starting with blank model 'en'
Counting training words (limit=0)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:632: UserWarning: [W022] Training a new part-of-speech tagger using a model with no lemmatization rules or data. This means that the trained model may not be able to lemmatize correctly. If this is intentional or the language you're using doesn't have lemmatization data, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed.
  proc.begin_training(
/usr/local/lib/python3.8/dist-packages/spacy/language.py:632: UserWarning: [W033] Training a new part-of-speech tagger using a model with no lexeme normalization table. This may degrade the perfor

Notice how the performance improves with each iteration!
## Evaluating the model with test data set (`spacyNER_data/test.json`)

### On Trained model (`model/model-best`)

In [5]:
#create a folder to store the output and visualizations. 
!mkdir result
!python3.8 -m spacy evaluate model/model-best spacyNER_data/test.json -dp result
# !python -m spacy evaluate model/model-final data/test.txt.json -dp result


================================== Results ==================================

Time      40.14 s
Words     46666  
Words/s   1163   
TOK       100.00 
POS       95.17  
UAS       0.00   
LAS       0.00   
NER P     81.39  
NER R     81.71  
NER F     81.55  
Textcat   0.00   

/usr/local/lib/python3.8/dist-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
✔ Generated 25 parses as HTML
result


a Visualization of the entity tagged test data can be seen in result/entities.html folder. 

### On spacy's Pretrained NER model (`en`)

In [8]:
#!mkdir pretrained_result
!python3.8 -m spacy evaluate en spacyNER_data/test.json -dp pretrained_result

Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.8/dist-packages/spacy/__main__.py", line 33, in <module>
    plac.call(commands[command], sys.argv[1:])
  File "/usr/local/lib/python3.8/dist-packages/plac_core.py", line 367, in call
    cmd, result = parser.consume(arglist)
  File "/usr/local/lib/python3.8/dist-packages/plac_core.py", line 232, in consume
    return cmd, self.func(*(args + varargs + extraopts), **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/spacy/cli/evaluate.py", line 49, in evaluate
    nlp = util.load_model(model)
  File "/usr/local/lib/python3.8/dist-packages/spacy/util.py", line 175, in load_model
    raise IOError(Errors.E050.format(name=name))
OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python pack

a Visualization of the entity tagged test data can be seen in pretrained_result/entities.html folder. 